In [ ]:

#include <stdio.h>
#include <xc.h>
#define _XTAL_FREQ 20000000
#define Trigger RB3 // is Trigger
#define Echo RB4// is Echo
#pragma config FOSC = HS        // Oscillator Selection bits (HS oscillator)
#pragma config WDTE = OFF       // Watchdog Timer Enable bit (WDT disabled)
#pragma config PWRTE = ON       // Power-up Timer Enable bit (PWRT enabled)
#pragma config BOREN = ON       // Brown-out Reset Enable bit (BOR enabled)
#pragma config LVP = OFF        // Low-Voltage (Single-Supply) In-Circuit Serial Programming Enable bit (RB3 is digital I/O, HV on MCLR must be used for programming)
#pragma config CPD = OFF        // Data EEPROM Memory Code Protection bit (Data EEPROM code protection off)
#pragma config WRT = OFF        // Flash Program Memory Write Enable bits (Write protection off; all program memory may be written to by EECON control)
#pragma config CP = OFF         // Flash Program Memory Code Protection bit (Code protection off)

int time_taken;
float distance;
int sensor_free;
int track_1;

void forward();
void right();
void left();
void back_off();
void stop();
void backward_right();
void backward_left();
void calculate_distance(void);
void obstacle_detection(void);
void on_line (void);

void main(void)
 {

   TRISD2 = 1; TRISD3 = 1; //Bath the IR sensor pins are declared as input
   TRISC4 = 0; TRISC5 = 0; //Motor 1 pins declared as output
   TRISC6 = 0; TRISC7 = 0; //Motor 2 pins declared as output
   TRISB4 = 1; //Front IR sensor is set as input pin
   TRISB5 = 1; //Left IR sensor is set as input pin
   TRISB6 = 1; //Right IR sensor pins is set as input

   while(1)
   {
       obstacle_detection();
       if (sensor_free == 0)  //no obstacle in front
       {
	        on_line();
           if(track_1 == 0) //robot is on the proper path
           {
               forward();
           }

           else if (track_1 == 1) //left sensor overlaps with black line
           {
               do
               {
                left();
               } while(track_1 == 1);
           }
           else if (track_1 == 2) //right sensor overlaps with black line
           {
               do
               {
                right();
               } while(track_1 == 2);
           }
           else
           {
               stop();
           }
       }
       else if (sensor_free == 1)
       {
           stop();
           __delay_ms(25);
           backward_left();
           stop();
       }
       else if (sensor_free == 2)
       {
           stop();
           __delay_ms(25);
           backward_right();
           stop();
       }
       else if (sensor_free == 3)
       {
           stop();
       }
   }
 }

 void forward()
   {
      RC4=0; RC5=1; //Motor 1 forward
      RC6=1; RC7=0; //Motor 2 forward
   }

void right()
   {
       RC4=1; RC5=1; //Motor 1 stop
       RC6=1; RC7=0; //Motor 2 forward
   }

void left()
   {
      RC4=0; RC5=1; //Motor 1 forward
      RC6=1; RC7=1; //Motor 2 stop
   }

void stop()
   {
      RC4=1; RC5=1; //Motor 1 stop
      RC6=1; RC7=1; //Motor 2 stop
   }

void back_off() //used to drive the robot backward
    {
      RC4=1; RC5=0; //Motor 1 reverse
      RC6=0; RC7=1; //Motor 2 reverse
      __delay_ms(25);
    }

void backward_right()
    {
        back_off();
        right();    //turns 90 degrees right
        __delay_ms(25);
	    stop();
	    forward();
	    __delay_ms(50);
	    stop();
	    left();   //turns 90 degrees left
	    __delay_ms(25);
	    stop();
	    while(RB6 ==0)  //goes forward till it detects object which was in the path
	    {
	        forward();
	    }

	    while(RB6 ==1)    // goes past object which was in its path
	    {
	        forward();
	    }
	     __delay_ms(25);
	    stop();
	    left();
	    __delay_ms(25);
	    stop();
	    while(RD2==1 && RD3==1) //moves until it reaches the black line
	    {
	       forward();
	    }
	    stop();
	    right();
	    __delay_ms(25);
    }

void backward_left()
	 {
         back_off();
         left();        //turns 90 degrees left
         __delay_ms(25);
	    stop();
	    forward();
	    __delay_ms(25);
	    stop();
	    right();      //turns 90 degrees right
	    __delay_ms(25);
	    stop();
	    while(RB5=0){   //goes forward till it detects object which was in the path
	        forward();
	    }

	    while(RB5=1){
	        forward();  // goes past object which was in its path
	    }
	    __delay_ms(25);
	    stop();
	    right();   // turns right
	    __delay_ms(25);
	    stop();
	    while(RD2==1 && RD3==1) //moves until it reaches the black line
	    {
	       forward();
	    }
	    stop();
	    left();   //takes a 90 degree turn left to get back on track
	    __delay_ms(25);
    }


void obstacle_detection(void) //function to detect obstacles
{
    if (RB4 == 0) //no obstacle in front
    {
        sensor_free = 0;
    }
    else if (RB4 == 1)  //obstacle in front
    {
        if (RB5 == 0)   //no obstacle on the left
        {
            sensor_free = 1;
        }
        else if (RB5 == 1 && RB6 ==0)   //obstacle on left, right is free
        {
            sensor_free = 2;
        }
        else if (RB5 == 1 && RB6 == 1)  //obstacle on both sides
        {
            sensor_free = 3;
        }

    }
}

void on_line (void)
{
    if (RD2==1 && RD3==1) //Both sensor not over black line
    {
       track_1= 0;
    }
    else if (RD2==0 && RD3==1) //Left sensor is over black line
    {
        track_1 = 1;
    }
    else if (RD2==1 && RD3==0) //Right sensor is over black line
    {
        track_1 = 2;
    }
    else //Both Sensor over black line
    {
        track_1 = 3;
    }

}
